# 01 Pickle

In [1]:
# Setup
import pickle
from datetime import datetime

In [2]:
# Solution part 1
from urllib.request import urlopen
from tempfile import TemporaryFile

tmp = TemporaryFile('w+b')
web = urlopen('http://example.com')
anon = lambda: None

unpickle = [tmp, web, anon]

In [3]:
# Solution part 2
class Person:
    "Plain class that holds file descriptor"
    def __init__(self, name, address, favorite_color):
        self.name = name
        self.address = address
        self.favorite_color = favorite_color
        self.created_at = datetime.now()

    def __setstate__(self, data):
        self.__dict__ = data
        self.created_at = datetime.now()
        
person =  Person('David', '45 Main St', "Red")

## Tests

In [4]:
def test_listed_types():
    from itertools import combinations
    assert len(unpickle) >= 3, "Provide at least 3 objects"
    for (a, b) in combinations(unpickle, 2):
        assert not issubclass(type(a), type(b)), \
            f"issubclass({a}, {b}) not permitted"
        assert not issubclass(type(b), type(a)), \
            f"issubclass({b}, {a}) not permitted"
        
test_listed_types()

In [5]:
def test_pickle_failure():
    for obj in unpickle:
        pickled = False
        try:
            pkl = pickle.dumps(obj)
            pickled = True
        except:
            pass
            
        if pickled:
            assert False, f"{repr(obj)} should not be pickleable"

            
test_pickle_failure()

In [6]:
def test_person_attributes():
    assert hasattr(person, 'name')
    assert hasattr(person, 'address')
    assert hasattr(person, 'favorite_color')
    assert hasattr(person, 'created_at')
    assert isinstance(person.created_at, datetime)

test_person_attributes()

In [7]:
def test_deserialize():
    pkl = pickle.dumps(person)
    t1 = datetime.now()
    new = pickle.loads(pkl)
    t2 = datetime.now()
    assert person.created_at < t1 < new.created_at < t2, \
        "Time must be refreshed when record is deserialized"
    assert person.name == new.name
    assert person.address == new.address
    assert person.favorite_color == new.favorite_color
    
test_deserialize()

# 02 Basic JSON

In [8]:
# Setup
import json

class Person:
    def __init__(self, name=None, address=None, 
                       favorite_color=None):
        self.name = name
        self.address = address
        self.favorite_color = favorite_color
        
    def toJSON(self):
        return "JSON string here"
    
    def fromJSON(self, jstr):
        return "Return new instance"

Red = "Red"
person =  Person('David', '45 Main St', Red)

In [9]:
bad_json = """
{"name": "David",
 "address": '45 Main St',
 "favorite_color": Red,
}
"""
good_json = bad_json

In [10]:
# Solution part 1
good_json = """
{"name": "David",
 "address": "45 Main St",
 "favorite_color": "Red"
}
"""

In [11]:
# Solution part 2
_Person = Person

class Person(_Person):
    def toJSON(self):
        return json.dumps(self.__dict__)
    
    def fromJSON(self, jstr):
        new = self.__class__()
        new.__dict__ = json.loads(jstr)
        return new
    
person =  Person('David', '45 Main St', Red)

## Tests

In [14]:
def test_json_fixed():
    try:
        person = json.loads(good_json)
    except:
        person = dict()
    assert set(person.keys()) == {'name', 'address', 'favorite_color'}
    assert person.get('name') == 'David'
    assert person.get('address') == "45 Main St"
    assert person.get('favorite_color') == Red
    
test_json_fixed()

In [13]:
def test_round_trip():
    from random import randint
    name = f"Jane-{randint(1, 1000)}"
    person = Person(name, "123 Any Road", "Green")
    jstr = person.toJSON()
    try:
        json.loads(jstr)
    except json.JSONDecodeError:
        assert False, "Invalid JSON string produced"
    new = person.fromJSON(jstr)
    assert isinstance(new, Person), f"Not a Person: {repr(new)}"
    assert person.name == new.name
    
test_round_trip()